In [1]:
from datasets import load_dataset

import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import torch.nn as nn

from torchvision import transforms

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("martingrzzler/kanjis2radicals")
""" Item in ds['train']
 Dataset({
    features: ['kanji_image', 'meta'],
    num_rows: 2027
})

{'kanji_image': Image(mode=None, decode=True, id=None), 
 'meta': {'id': Value(dtype='int32', id=None), 
          'characters': Value(dtype='string', id=None), 
          'meanings': Value(dtype='string', id=None), 
          'radicals': Sequence(feature={'characters': Value(dtype='string', id=None), 
                                        'id': Value(dtype='int32', id=None), 
                                        'slug': Value(dtype='string', id=None)
                                        }, length=-1, id=None)
         }
}
"""
# Split the dataset into train and test (80-20 split by default)
ds_split = ds['train'].train_test_split(test_size=0.2)

# Access the train and test subsets
ds_train = ds_split['train']
ds_test = ds_split['test']

In [3]:
# Extract all unique radicals
unique_radicals = set()
for element in ds['train']:
    radical_ids = element['meta']['radicals']['id']
    unique_radicals.update(radical_ids)

# Sort the unique radicals
sorted_unique_radicals = sorted(unique_radicals)

# Count total unique radicals
total_unique_radicals = len(sorted_unique_radicals)

print(f"Total unique radicals: {total_unique_radicals}")
print(f"Sorted unique radicals: {sorted_unique_radicals}")

Total unique radicals: 478
Sorted unique radicals: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

In [4]:
def label_vector(ds):
    # Crear un mapeo de radicales únicos a índices
    radical_to_index = {radical: idx for idx, radical in enumerate(unique_radicals)}

    # Transformar los identificadores de radicales en vectores binarios
    def generate_label_vector(radical_ids, total_radicals):
        label_vector = [0] * total_radicals
        for radical_id in radical_ids:
            if radical_id in radical_to_index:
                label_vector[radical_to_index[radical_id]] = 1
        return label_vector

    # Generar labels para el dataset de entrenamiento
    total_unique_radicals = len(unique_radicals)
    labels = []

    for element in ds:
        radical_ids = element['meta']['radicals']['id']
        label_vector = generate_label_vector(radical_ids, total_unique_radicals)
        labels.append(label_vector)
    return labels


In [5]:
train_labels = label_vector(ds_train)
test_labels = label_vector(ds_test)

In [6]:
image_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to a fixed size
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] for RGB
])

In [7]:
def sample_batch(ds_image, ds_label):
    # Define a transformation pipeline for the images
    batch_images = []
    batch_labels = []

    for i, _ in enumerate(ds_image):
        # Example image path and label (replace with actual paths)
        radical_ids = ds_label[i]
        
        # Load and transform the image
        image = ds_image[i]['kanji_image'].convert("RGB")
        image_tensor = image_transform(image)
        
        # Convert radical IDs to tensor
        label_tensor = torch.tensor(radical_ids, dtype=torch.long)
        
        # Append to batch lists
        batch_images.append(image_tensor)
        batch_labels.append(label_tensor)

    # Stack into a batch (list of tensors to tensor batch)
    batch_images = torch.stack(batch_images)
    batch_labels = torch.nn.utils.rnn.pad_sequence(batch_labels, batch_first=True, padding_value=-1)

    print(f"Batch Images Shape: {batch_images.shape}")  # [batch_size, channels, height, width]
    print(f"Batch Labels Shape: {batch_labels.shape}")  # [batch_size, max_seq_len]
    return batch_images, batch_labels


In [8]:

train_img, train_labels = sample_batch(ds_train, train_labels)
test_img, test_labels = sample_batch(ds_test, test_labels)

Batch Images Shape: torch.Size([1621, 3, 128, 128])
Batch Labels Shape: torch.Size([1621, 478])
Batch Images Shape: torch.Size([406, 3, 128, 128])
Batch Labels Shape: torch.Size([406, 478])


In [9]:
# Define the collate_fn to pad sequences and stack the images
def collate_fn(batch):
    images = torch.stack([item[0] for item in batch])  # Stack all images in the batch
    labels = [item[1] for item in batch]  # Get the labels (radical IDs) for each image
    
    # Pad the label sequences
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-1)
    
    return images, labels_padded

In [10]:
class KanjiRadicalDataset(Dataset):
    def __init__(self, images, labels):
        """
        Args:
            images (list): List of file paths to the Kanji images.
            labels (list): List of corresponding radical ID lists for each image.
        """
        self.images = images
        self.labels = labels

    def __len__(self):
        # Return the number of samples
        return len(self.images)

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the sample to retrieve.
        
        Returns:
            dict: A dictionary with 'image' and 'label' tensors.
        """
        image = self.images[idx]
        label = self.labels[idx]
        
        return image, label


In [11]:
# Create DataLoader for training and testing
train_dataset = KanjiRadicalDataset(images=train_img, labels=train_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=lambda batch: collate_fn(batch))

test_dataset = KanjiRadicalDataset(images=test_img, labels=test_labels)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=lambda batch: collate_fn(batch))


In [12]:
class conv_block_nested(nn.Module):

    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output

In [13]:
class Nested_UNet(nn.Module):
    def __init__(self, in_ch=3, out_ch=1):
        super(Nested_UNet, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Sequential(
            nn.Conv2d(filters[0], out_ch, kernel_size=1),
            nn.AdaptiveAvgPool2d((1, 1))
        )

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)
        output = output.view(output.size(0), -1)  # [batch_size, out_ch]
        return output


In [14]:

# class ImageToRadicalsModel(nn.Module):
#     def __init__(self, num_radicals):
#         super(ImageToRadicalsModel, self).__init__()
        
#         # Define the convolutional layers to process the image
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
#         self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
#         self.fc1 = nn.Linear(256 * 16 * 16, 512)  # Adjust based on input size and feature maps
        
#         # Define an LSTM or GRU for sequence generation
#         self.rnn = nn.LSTM(input_size=512, hidden_size=256, batch_first=True)
        
#         # Output layer
#         self.output = nn.Linear(256, num_radicals)  # num_radicals is the size of your output space
    
#     # Modificación del forward sin secuencias
#     def forward(self, x):
#         # Pasar por las capas convolucionales
#         x = self.conv1(x)
#         x = torch.relu(x)
#         x = self.conv2(x)
#         x = torch.relu(x)
#         x = self.conv3(x)
#         x = torch.relu(x)
        
#         # Aplanar para la capa totalmente conectada
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
        
#         # Pasar directamente por la capa de salida
#         x = self.output(x)
#         return x  # Output: [batch_size, num_radicals]


In [22]:
from tqdm import tqdm  # Importar tqdm para la barra de progreso

# Model, Loss, Optimizer
num_radicals = 478  # Total unique radicals in the dataset
model = Nested_UNet(3, num_radicals)
criterion = nn.BCEWithLogitsLoss()  # Multi-label classification
optimizer = Adam(model.parameters(), lr=1e-4)

# Training Loop
for epoch in range(50):  # Number of epochs
    model.train()
    epoch_loss = 0  # Para calcular la pérdida acumulada por epoch

    # Barra de progreso para batches
    with tqdm(train_loader, desc=f"Epoch {epoch+1}") as pbar:
        for batch in pbar:
            images, targets = batch  # Images: [batch_size, 3, 128, 128], Targets: [batch_size, num_radicals]
            optimizer.zero_grad()

            # Forward pass
            predictions = model(images)  # Output: [batch_size, num_radicals]

            # Calculate loss
            loss = criterion(predictions, targets.float())  # Convert targets to float for BCEWithLogitsLoss
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix({"Loss": loss.item()})  # Actualiza la barra de progreso con la pérdida actual
    
    print(f"Epoch {epoch+1}, Average Loss: {epoch_loss / len(train_loader)}")


Epoch 1: 100%|██████████| 102/102 [1:01:10<00:00, 35.99s/it, Loss=0.552]


Epoch 1, Average Loss: 0.6263906885595882


Epoch 2: 100%|██████████| 102/102 [57:12<00:00, 33.65s/it, Loss=0.414]


Epoch 2, Average Loss: 0.47836516271619234


Epoch 3: 100%|██████████| 102/102 [56:22<00:00, 33.16s/it, Loss=0.317]


...

Epoch 48, Average Loss: 0.020335925487326642


Epoch 49: 100%|██████████| 102/102 [56:05<00:00, 32.99s/it, Loss=0.0231]


Epoch 49, Average Loss: 0.019842500579269493


Epoch 50: 100%|██████████| 102/102 [52:26<00:00, 30.85s/it, Loss=0.0213]

Epoch 50, Average Loss: 0.01948562264442444


In [23]:
# Test Loop
model.eval()  # Set the model to evaluation mode
test_loss = 0  # To accumulate the loss
all_predictions = []  # To store all predictions
all_targets = []  # To store all true labels

with torch.no_grad():  # Disable gradient computation
    with tqdm(test_loader, desc="Testing") as pbar:  # Use tqdm for progress
        for batch in pbar:
            images, targets = batch  # Images: [batch_size, 3, 128, 128], Targets: [batch_size, num_radicals]

            # Forward pass
            predictions = model(images)  # Output: [batch_size, num_radicals]

            # Calculate loss
            loss = criterion(predictions, targets.float())
            test_loss += loss.item()

            # Store predictions and targets for further analysis
            all_predictions.append(torch.sigmoid(predictions))  # Apply sigmoid for probabilities
            all_targets.append(targets)

            # Update tqdm with loss information
            pbar.set_postfix({"Loss": loss.item()})

# Calculate average test loss
avg_test_loss = test_loss / len(test_loader)
print(f"Average Test Loss: {avg_test_loss}")

# Combine all predictions and targets
all_predictions = torch.cat(all_predictions, dim=0).cpu()
all_targets = torch.cat(all_targets, dim=0).cpu()

# Example: Analyze results (optional)
# Convert predictions to binary (threshold=0.5)
binary_predictions = (all_predictions > 0.5).float()


Testing: 100%|██████████| 26/26 [04:39<00:00, 10.73s/it, Loss=0.0216]

Average Test Loss: 0.024856121709140446


In [38]:
import torch
from torchvision import transforms
from PIL import Image

# Configuración
model.eval()  # Ponemos el modelo en modo de evaluación
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Transformaciones para la imagen de entrada (ajusta según tu entrenamiento)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Asegurarse de que el tamaño coincida con el de entrenamiento
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalizar (ajusta según tu dataset)
])

predicted_labels = []
with tqdm(test_loader, desc="Testing") as pbar:  # Use tqdm for progress
    for batch in pbar:
        images, targets = batch  # Images: [batch_size, 3, 128, 128], Targets: [batch_size, num_radicals]

        # Enviar la imagen al dispositivo
        image_tensor = images.to(device)

        # Inferencia
        with torch.no_grad():
            predictions = model(image_tensor)  # Salida: [1, num_radicals]
            predictions = torch.sigmoid(predictions)  # Aplicar sigmoide para convertir a probabilidades

        # Procesar las salidas
        threshold = 0.5  # Define un umbral para clasificar
        predicted_labels.append((predictions > threshold).int())  # [1, num_radicals]


Testing: 100%|██████████| 26/26 [05:33<00:00, 12.81s/it]


In [53]:
print(predicted_labels[3])

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)


In [52]:
print(all_targets[3])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,